Notebook generates Fig. 1.

Written 2023-07-06 by W. Sauthoff (sauthoff@mines.edu)

# Setup computing environment

In [ ]:
# Install dependency not pre-installed in CryoCloud
%pip install openpyxl --quiet

In [ ]:
# Import packages
import earthaccess
import geopandas as gpd
from math import radians
from matplotlib.colors import LogNorm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import os
import pandas as pd
from pyproj import CRS, Transformer
import rioxarray
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import xarray as xr

# Magic functions
%matplotlib widget
# %matplotlib inline

# Update the default font to Arial (and add backup if not found)
plt.rcParams['font.family'] = ['sans-serif']

# Define data directories dependent on home environment
if os.getenv('HOME') == '/home/jovyan':
    DATA_DIR = '/home/jovyan/data'
    OUTPUT_DIR = '/home/jovyan/1_evolving_lakes/output'

# Define utility function
def ll2ps(lon, lat):
    """
    Transform coordinates from geodetic coordinates (lon, lat)
    to Antarctic Polar Stereograph coordinates (x, y)
    x, y = ll2ps(lon, lat)
    """
    crs_ll = CRS("EPSG:4326")
    crs_xy = CRS("EPSG:3031")
    ll_to_xy = Transformer.from_crs(crs_ll, crs_xy, always_xy = True)
    x, y = ll_to_xy.transform(lon, lat)
    return x, y

In [ ]:
def filter_gdf_by_folder_contents(gdf, folder_path, exclude=True, prefix=None, suffix=None, suffix_pattern=None, file_extension=None):
    '''
    Filter the GeoDataFrame based on processed lake names from the folder contents.
    
    Args:
    gdf: GeoDataFrame to be filtered.
    folder_path: Path to the directory containing files and/or subdirectories.
    exclude: If True, excludes rows where the 'name' is in the folder_path directories or files.
             If False, includes only rows where the 'name' is in the folder_path directories or files.
    prefix: Optional string to remove from the beginning of filenames.
    suffix: Optional string to remove from the end of filenames.
    suffix_pattern: Optional regex pattern to remove from the end of filenames.
    file_extension: Optional string specifying the file extension to filter (e.g., 'png', 'txt').
    
    Returns:
    GeoDataFrame filtered based on the presence of 'name' in folder_path.

    # Example usage:
    remaining_lakes = filter_gdf_by_folder_contents(
        stationary_lakes_gdf, 
        folder_path,
        prefix='plot_evolving_outlines_time_series_', 
        suffix_pattern=r'\d+\.\d+m-level_\d+x-with',
        file_extension='txt'
    )
    '''
    def process_name(name):
        """Helper function to remove prefix and suffix from a name"""
        processed_name = name
        
        # First strip the file extension if it exists
        processed_name = os.path.splitext(processed_name)[0]
        
        if prefix and processed_name.startswith(prefix):
            processed_name = processed_name[len(prefix):]
            
        if suffix_pattern:
            processed_name = re.sub(suffix_pattern + '$', '', processed_name)
        elif suffix and processed_name.endswith(suffix):
            processed_name = processed_name[:-len(suffix)]
            
        return processed_name.lower().strip()
    
    # Get all files and filter by extension if specified
    all_files = os.listdir(folder_path)
    if file_extension:
        clean_extension = file_extension.lstrip('.')
        all_files = [f for f in all_files if f.lower().endswith(f'.{clean_extension.lower()}')]
    
    # Process filenames to get lake names
    names_in_folder = {
        process_name(name)
        for name in all_files
    }
    
    # Filter without adding and then dropping a new column
    gdf_filtered = gdf[gdf['name'].str.lower().str.strip().apply(
        lambda x: (x not in names_in_folder) if exclude else (x in names_in_folder)
    )]
    
    return gdf_filtered.reset_index(drop=True)

# Import datasets

In [ ]:
# Find cloud-hosted MEaSUREs Phase-Based Antarctica Ice Velocity Map, Version 1
# DOI from https://nsidc.org/data/NSIDC-0754/versions/1
results = earthaccess.search_data(
    doi='10.5067/PZ3NJ5RXRH10',
    cloud_hosted=True,
    bounding_box=(1, -89, -1, -89)  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
)

In [ ]:
# Open data granules as s3 files to stream
files = earthaccess.open(results)
files

In [ ]:
# Open each file, which are quadrants in polar stereographic coordinations around the Geographic South Pole
ice_vel = xr.open_dataset(files[0])
ice_vel

In [ ]:
# Specify the variables to keep
variables_to_keep = ['x', 'y', 'VX', 'VY']

variables_to_drop = [var for var in ice_vel.variables if var not in variables_to_keep]

# Drop variables to reduce memory consumption
ice_vel = ice_vel.drop_vars(variables_to_drop)
ice_vel

In [ ]:
# Calculate velocity magnitude
vel_mag = (ice_vel['VX']**2 + ice_vel['VY']**2)**0.5

# Delete intermediary variable to conserve memory
del ice_vel

In [ ]:
vel_mag.max()

In [ ]:
10**3

In [ ]:
# Scripps Grounding Line (Depoorter and others, 2013)
# https://doi.pangaea.de/10.1594/PANGAEA.819147
Scripps_gl = gpd.read_file(DATA_DIR + '/boundaries/Depoorter2013/Antarctica_masks/scripps_antarctica_polygons_v1.shp')

# Isolate land ice and ice shelf
Scripps_landice = Scripps_gl[Scripps_gl['Id_text'] == 'Grounded ice or land']
Scripps_iceshelf = Scripps_gl[Scripps_gl['Id_text'] == 'Ice shelf']
Scripps_icerise = Scripps_gl[Scripps_gl['Id_text'] == 'Ice rise or connected island']

In [ ]:
# Import stationary subglacial lake outlines
stationary_lakes_gdf = gpd.read_file(os.path.join(os.getcwd(), 'output/lake_outlines/stationary_outlines/stationary_outlines_gdf.geojson'))

In [ ]:
# Import CryoSat-2 SARIn mode mask
# See 0_preprocess_data.ipynb for data source and pre-processing steps
gdf_SARIn_3_1 = gpd.read_file('output/CryoSat2_SARIn_mode_masks/gdf_SARIn_3_1.geojson')
gdf_SARIn_3_1_3_6_diff= gpd.read_file('output/CryoSat2_SARIn_mode_masks/gdf_SARIn_3_1_3_6_diff.geojson')

# Fig. 1

In [ ]:
# Create filtered geodataframes of lakes based on whether they have evolving outlines
folder_path = os.path.join ('output/lake_outlines/evolving_outlines')

# Lakes with non-dynamic outlines (.txt)
no_evolving_outlines_lakes = filter_gdf_by_folder_contents(stationary_lakes_gdf, folder_path, file_extension='txt', exclude=False)
print('non-dynamic:',len(no_evolving_outlines_lakes))

# Lakes with evolving outlines (.geojson)
evolving_outlines_lakes = filter_gdf_by_folder_contents(stationary_lakes_gdf, folder_path, file_extension='geojson', exclude=False)

# For the evolving_outlines_lakes, we must add the special case of Site_B_Site_C that are now a combined lake
include_list = ['Site_B', 'Site_C']
included_rows = stationary_lakes_gdf[stationary_lakes_gdf['name'].isin(include_list)]
evolving_outlines_lakes = pd.concat([evolving_outlines_lakes, included_rows]).drop_duplicates()
print('dynamic:',len(evolving_outlines_lakes))

In [ ]:
# Combine both sets of lakes that meet either criterion
valid_lakes = pd.concat([evolving_outlines_lakes, no_evolving_outlines_lakes]).drop_duplicates(subset='name')

# Now filter out the valid lakes to find the ones that don't satisfy either criterion
missing_lakes = stationary_lakes_gdf[~stationary_lakes_gdf['name'].isin(valid_lakes['name'])].reset_index(drop=True)
missing_lakes

In [ ]:
stationary_lakes_gdf['CS2_SARIn_start'].unique()

In [ ]:
# Step 1: filter out Crane_Glacier
filtered_gdf = stationary_lakes_gdf[stationary_lakes_gdf['name'] != 'Crane_Glacier']

# Step 2a: count where CS2_SARIn_start is 2010.5 or 2013.75
count_CS2 = filtered_gdf['CS2_SARIn_start'].isin(['2010.5', '2013.75']).sum()

# Step 2b: count where CS2_SARIn_start == <NA>
count_IS2 = filtered_gdf['CS2_SARIn_start'].isin(['<NA>']).sum()

print("Rows with CS2_SARIn_start = 2010.5 or 2013.75:", count_CS2)
print("Rows with CS2_SARIn_start = <NA>:", count_IS2)

In [ ]:
# Get the bounds of the CryoSat-2 SARIn mode mask GeoDataFrame
bounds = gdf_SARIn_3_1.geometry.bounds

# Extract the min and max x and y values from the bounds
x_min, x_max = bounds['minx'].min(), bounds['maxx'].max()
y_min, y_max = bounds['miny'].min(), bounds['maxy'].max()
x_buffer, y_buffer = (x_max-x_min)*0.01, (y_max-y_min)*0.01, 

# Create fig and axes objects
fig, ax = plt.subplots(figsize=(10,10))

# Plot ice-surface velocity
# cax = vel_mag.plot(ax=ax, cmap='Greys_r', norm=LogNorm(vmin=vel_mag.min(), vmax=vel_mag.max()), zorder=2)
cax = vel_mag.plot(ax=ax, cmap='Greys_r', norm=LogNorm(vmin=1, vmax=1000), zorder=1, add_colorbar=False)

# Plot Scripps grounding line
# Use black facecolor to fill in gaps in velocity array
Scripps_landice.boundary.plot(ax=ax, linewidth=0.25, edgecolor='k', facecolor='k', zorder=0) 
Scripps_landice.boundary.plot(ax=ax, linewidth=0.25, edgecolor='k', facecolor=None, zorder=2) 
Scripps_iceshelf.boundary.plot(ax=ax, linewidth=0.25, edgecolor='k', facecolor=None, zorder=2) 
Scripps_icerise.boundary.plot(ax=ax, linewidth=0.25, edgecolor='k', facecolor=None, zorder=2) 

# Plot CryoSat-2 SARIn mode masks
gdf_SARIn_3_1.plot(ax=ax, edgecolor='blue', facecolor='blue', alpha=0.1, zorder=3)
gdf_SARIn_3_1.boundary.plot(ax=ax, color='blue', linewidth=0.5, zorder=3)
gdf_SARIn_3_1_3_6_diff.plot(ax=ax, edgecolor='blue', facecolor='blue', alpha=0.1, zorder=3)
gdf_SARIn_3_1_3_6_diff.boundary.plot(ax=ax, color='blue', linewidth=0.5, zorder=3)
CS2_SARIn_union_polygon = unary_union([gdf_SARIn_3_1.geometry[0], gdf_SARIn_3_1_3_6_diff.geometry[0]])
CS2_SARIn_union_gdf = gpd.GeoDataFrame({'id': [1], 'geometry': [CS2_SARIn_union_polygon]}, crs='EPSG:3031')
CS2_SARIn_union_gdf = CS2_SARIn_union_gdf.to_crs(epsg=4326)

# Create ICESat-2 pole hole geodataframe
center_x, center_y = 0, 0  # Geographic South Pole
x, y = ll2ps(0, -88)
radius = y  # Radius from Geographic South Pole to -88 S latitude (ICESat-2's southernmost coverage)
del x, y
IS2_pole_hole_poly = Point(center_x, center_y).buffer(radius)
IS2_pole_hole_gdf = gpd.GeoDataFrame(index=[0], geometry=[IS2_pole_hole_poly], crs="EPSG:3031")
IS2_pole_hole_gdf = IS2_pole_hole_gdf.to_crs(epsg=4326)

# IS2 interior to CS2 SARIn mode coverage
IS2_interior_poly = Polygon(shell=CS2_SARIn_union_gdf.geometry[0].interiors[0].coords, 
                            holes=[IS2_pole_hole_gdf.geometry[0].exterior.coords])
IS2_interior_gdf = gpd.GeoDataFrame({'id': [1], 'geometry': [IS2_interior_poly]}, crs='EPSG:4326')
IS2_interior_gdf = IS2_interior_gdf.to_crs(epsg=3031)
IS2_interior_gdf.plot(ax=ax, edgecolor='green', facecolor='green', alpha=0.1, zorder=3)
IS2_interior_gdf.boundary.plot(ax=ax, color='green', linewidth=0.5, zorder=3)

# Overlay previously identified active subglacial lake stationary outlines
stationary_lakes_color = 'turquoise'
# stationary_lakes_gdf.boundary.plot(ax=ax, facecolor=stationary_lakes_color, 
#     edgecolor=stationary_lakes_color, linewidth=1, alpha=1, zorder=4)
evolving_outlines_lakes.boundary.plot(ax=ax, facecolor=stationary_lakes_color, 
    edgecolor=stationary_lakes_color, linewidth=1, alpha=1, zorder=4)
no_evolving_outlines_lakes.boundary.plot(ax=ax, facecolor='red', 
    edgecolor='red', linewidth=1, alpha=1, zorder=4)

# Create patches for the two lake types
evolving_lakes_patch = mpatches.Patch(
    facecolor=stationary_lakes_color,
    edgecolor=stationary_lakes_color,
    label='Evolving outlines found'
)
no_evolving_outlines_lakes_patch = mpatches.Patch(
    facecolor='red',
    edgecolor='red',
    label='Evolving outlines not found'
)

# Create patches for the other elements
cs2_patch = mpatches.Patch(
    facecolor=(0, 0, 1, 0.1),   # blue with 0.1 opacity
    edgecolor='blue',
    # alpha=0.1,
    # label='CryoSat-2 SARIn and ICESat-2 coterminous coverage'
)
is2_patch = mpatches.Patch(
    facecolor=(0, 1, 0, 0.1),   # green with 0.1 opacity
    edgecolor='green',
    linewidth=0.75,
    # alpha=0.1,
    # label='ICESat-2 coverage (CryoSat-2 SARIn unavailable)'
)

# Create the legend with grouping
legend1 = ax.legend(
    [evolving_lakes_patch, 
     no_evolving_outlines_lakes_patch],
    [f'Evolving outlines found (n={len(evolving_outlines_lakes)})', 
     f'Evolving outlines not found (n={len(no_evolving_outlines_lakes)})'],
    title='Re-examined active subglacial lakes',
    loc='upper left',
    bbox_to_anchor=(0, 0.225),
    alignment='left',
    frameon=False,
)

# Add the second legend for the coverage areas
ax.add_artist(legend1)  # This keeps the first legend visible
ax.legend(
    [cs2_patch, 
     is2_patch],
    [f'CryoSat-2 SARIn and ICESat-2 (n={count_CS2})', 
     f'ICESat-2 only (n={count_IS2})'],
    title='Satellite/mode coverage',
    loc='upper left',
    bbox_to_anchor=(0, 0.118),
    alignment='left',
    frameon=False,
)

# Change polar stereographic m to km
km_scale = 1e3
ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
ax.xaxis.set_major_formatter(ticks_x)
ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
ax.yaxis.set_major_formatter(ticks_y)  

# Label axes
ax.set_xlabel('x [km]', size=15)
ax.set_ylabel('y [km]', size=15)

# Set axes limits
ax.set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

# Plot colorbar
axins = inset_axes(ax, width="50%", height="2%", loc='upper left',
                   bbox_to_anchor=(0.07, -0.05, 0.5, 1),  # [left, bottom, width, height]
                   bbox_transform=ax.transAxes,
                   borderpad=0)
fig.colorbar(cax, cax=axins, label='ice velocity [m a$^{-1}$]', orientation='horizontal', extend='max')

# Save and close plot
plt.savefig(OUTPUT_DIR + '/Fig1_subglacial_lake_distribution.jpg', dpi=300, bbox_inches='tight')

# Preview plot
plt.show()

In [ ]:
plt.close('all')